In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader

In [59]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

In [60]:
def load_data(root_dir):
    root_path = Path(root_dir)

    # 클래스(레이블) 생성
    classes = sorted([d.name for d in root_path.iterdir() if d.is_dir()])
    class_to_idx = {cls_name:idx for idx, cls_name in enumerate(classes)}
    idx_to_class = {idx: cls_name for cls_name, idx in class_to_idx.items()}

    # 이미지 파일 수집
    images = []
    labels = []
    for cls_name in classes:
        cls_dir = root_path / cls_name
        for img_path in cls_dir.glob("*.jpg"):
            images.append(img_path)
            labels.append(class_to_idx[cls_name])
    return images, labels, class_to_idx, idx_to_class

In [61]:
def split_data(images, labels, train_ratio=0.8):
    n_sample = len(images)
    n_train = int(n_sample * train_ratio)

    indices = np.random.permutation(n_sample)
    train_indices = indices[:n_train]
    val_indices = indices[n_train:]

    train_images = [images[i] for i in train_indices]
    val_images = [images[i] for i in val_indices]
    train_labels = [labels[i] for i in train_indices]
    val_labels = [labels[i] for i in val_indices]

    return train_images, val_images, train_labels, val_labels

In [62]:
class FruitDataset(Dataset):
    # 과일 이미지 데이터셋
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = self.images[index]
        label = self.labels[index]

        image = Image.open(img_path).convert("RGB")

        return image, label

In [63]:
data_dir = "data/fruit" # 데이터 위치
num_epochs = 3 # 반복 
batch_size = 32 # 학습크기

torch.manual_seed(42) # 재현성을 위한 랜덤 값
np.random.seed(42) # 재현성을 위한 랜덤 값

device = get_device() # 계산 장치 설정
print(f"{device}를 사용합니다.")

cpu를 사용합니다.


In [67]:
images, labels, class_to_idx, idx_to_class = load_data(data_dir)
train_images, val_images, train_labels, val_labels = split_data(images, labels)
train_dataset = FruitDataset(train_images, train_labels)
val_dataset = FruitDataset(val_images, val_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=batch_size, shuffle=True)